Hereby I try to modulate JPA using it's hamiltonian. Damping coeff and Kerr nonlin. coeff required.

In [3]:
from matplotlib import pyplot as plt
from numpy import linspace
import numpy as np
import scipy
import scipy.optimize

In [8]:
#constants defenition
w_0 = 5.69396298026 *6.28e9
#K = -38278627.2593
K = -13841689.8521
kappa = 31.46858539 * 6.28e6
gamma = .1*kappa
#kappa - coupling, gamma - radiation losses.

In [11]:
#Defining pump photon number
def delta(w_p):
    return((w_p - w_0)/(kappa + gamma))

def Alpha_in_mod(x):
    return(x*kappa**0.5/(kappa + gamma))
def Ksi(x):
    return(K*x**2/(kappa + gamma))
    #return(K*x**2/2/(kappa + gamma))

for alpha_in in linspace(2e4, 3e4, 10):
    #alpha_in_mod = alpha_in*kappa**0.5/(kappa + gamma)
    ksi = Ksi(Alpha_in_mod(alpha_in))#K*alpha_in_mod**2/(kappa + gamma)

    def f(x,w_p):
        return( (delta(w_p)**2 + 0.25)*x - 2*delta(w_p)*ksi*x**2 + ksi**2*x**3 - 1 )
        #return( (delta(w_p)**2 + 0.25)*x - 2*delta(w_p)*ksi*x**2 + ksi**2*x**3 - 0.5 ) #2baths

    n_array = []
    for f_p in linspace(w_0 - 4*(gamma+kappa), w_0 + 1*(gamma+kappa), 1000):
        n_array.append(scipy.optimize.bisect(lambda x: f(x, f_p), 0,5, xtol=1e-17))

    bufer = np.array(n_array)
    f_p = linspace(w_0 - 4*(gamma+kappa), w_0 + 1*(gamma+kappa), 1000)

    plt.plot(delta(f_p), bufer)


#plt.xlim(-2,0)
plt.xlabel('delta')
plt.ylabel('n')
plt.title('Mean pump photon number')
plt.show()

In [12]:
#Linear signal response
#alpha is fond from prev graph
alpha = 2.8e4
ksi = Ksi(Alpha_in_mod(alpha))
#w_p = w_0 - 1.43*kappa
#for w_p in linspace(w_0 - 2*(gamma + kappa), w_0 - 0.7*(kappa + gamma), 100):
for w_p in linspace(w_0 - 1.0*(gamma + kappa), w_0 -.8*(kappa + gamma), 20):
    delt = (w_p - w_0)/(kappa + gamma)
    n = scipy.optimize.bisect(lambda x: f(x, w_p), 0,5, xtol=1e-17)
    print n

    w_real = linspace(-4*kappa,3*kappa,1e5)
    w = w_real/(kappa + gamma)
    lambda1 = 0.5 + ( (ksi*n)**2 - (delt - 2*ksi*n)**2  + 0*1j)**.5
    lambda2 = 0.5 - ( (ksi*n)**2 - (delt - 2*ksi*n)**2  + 0*1j)**.5

    g_s = -1 + 0*1j + kappa/(kappa + gamma) * (0.5 + 1j*(delt - 2*ksi*n - w)) / ((1j*w - lambda1)*(1j*w - lambda2))
    g_i = kappa/(kappa + gamma)*(-1j*ksi*n) / ((1j*w - lambda1)*(1j*w - lambda2))

    plt.plot(w, 20*np.log10(np.abs(g_s)))
    #plt.plot(np.abs(g_s)**2 - np.abs(g_i)**2)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
#plt.xlim(-2, 0)
plt.xlabel(r'\huge $\frac{w-w_p}{\kappa + \gamma}$')
plt.ylabel(r'Gain, dB')
plt.title(r'\huge \textbf{ $\gamma = 3\kappa$}')
plt.show()

1.26168837058
1.31251456114
1.36900820817
1.43262781603
1.5055431312
1.59122986022
1.69590336463
1.83292629644
2.0451254225
3.82682255235
3.89750703552
3.94005485759
3.96716695508
3.98430356259
3.99438120057
3.99916427213
3.99981139532
3.99712919287
3.99170445452
3.98397894749


In [11]:
#3d gain plot
G_s = []

w_length = 100
alpha_length = 100

alpha_start = 2e5
alpha_stop = 3e5

alpha_plot_arr = linspace(alpha_start,alpha_stop, alpha_length)
w_plot_arr = linspace(w_0 - 2*(gamma + kappa), w_0, w_length)
def alpha_from_number(k):
    return(alpha_start + (alpha_stop - alpha_start)*k/alpha_length)

def w_p_from_number(l):
    return(w_0 - 2*(kappa + gamma) + l*2*(gamma + kappa)/w_length)

for k in range(alpha_length):
    G_s.append([])
    for l in range(w_length):
        
        alpha = alpha_from_number(k)
        w_p = w_p_from_number(l)
        ksi = Ksi(Alpha_in_mod(alpha))
        delt = (w_p - w_0)/(kappa + gamma)
        n = scipy.optimize.bisect(lambda x: f(x, w_p), 0,5, xtol=1e-17)

        w_real = linspace(-10e6,10e6,1e5)
        w = w_real/(kappa + gamma)
        lambda1 = 0.5 + ( (ksi*n)**2 - (delt - 2*ksi*n)**2  + 0*1j)**.5
        lambda2 = 0.5 - ( (ksi*n)**2 - (delt - 2*ksi*n)**2  + 0*1j)**.5

        g_s = -1 + 0*1j + kappa/(kappa + gamma) * (0.5 + 1j*(delt - 2*ksi*n - w)) / ((1j*w - lambda1)*(1j*w - lambda2))
        g_i = kappa/(kappa + gamma)*(-1j*ksi*n) / ((1j*w - lambda1)*(1j*w - lambda2))
        G_s[k].append(20*np.log10(np.max(np.abs(g_s))))

     
plt.contourf((w_plot_arr - w_0)/(kappa + gamma),alpha_plot_arr, G_s, 100, alpha=0.75 )
plt.colorbar()

plt.ylabel('alpha')
plt.xlabel('(w_p - w_0)/(kappa + gamma)')
plt.title('Gain')
plt.show()
print np.max(G_s)
#print G_s

KeyboardInterrupt: 

In [5]:
 print 10*np.log10(w_0*1e-34*kappa/2) +30

-116.944554678


In [31]:
print 2*3.141596/4*2**2

6.283192


In [7]:
print 2*1e-23*4/6.28e10/1e-34

12.7388535032


In [23]:
print 10*np.log10(   Alpha_in_mod(3e5)**2 * kappa*1e-34 *w_0) + 30

-94.299220275


In [1]:
jc = 3.1415*1.4/4 #Ic from AB formula
d = 2 #JJ diameter
print jc
print 2*jc*3.1415*d**2/4

1.099525
6.908315575


In [33]:
c = (0.88 * 8.85e-12 * 32 * 1e-12 /90e-9)
print  c * 1e12, 'pF/um**2'
print c * 1e12 * 45*42
print 5.3/c/42/1e12

0.00276906666667 pF/um**2
5.233536
45.5714836011


In [11]:
print 6.9/2

3.45


In [3]:
c = (0.88 * 8.85e-12 * 4 * 1e-12 /300e-9)
#print  c * 1e12, 'pF/um**2'
print c * 1e15 * 6*33

20.56032


In [37]:
###Inductance of meander

w = 5e-4 #lengths in cm
t = 0.4e-4
def ind(l): #L in  uH
        return(0.02 * l * ( np.log(2*l/(w + t)) + 0.50049 + (w+t)/3l ))


a = 5e-4
b = 5e-4
d = 5e-4
h = 50e-4
N = 1.5

Ind = 2*ind(a) + 2*ind(b) + N*ind(h) + (N+1)*ind(d)
print Ind*1e3
print (Ind * 1e-6 * 1e-12)**(-0.5) / 6.28e9


0.585511833925
6.58070643987


In [46]:
a = 60.
N = 4.

print a/2/np.sin(3.14159/4/N)

153.775055079


In [48]:
Z0 = 50.
Zl = 20.
g = 0.2

A = g/(1-g**2)**0.5 * 2 * (Z0*Zl)**0.5/np.abs(Zl - Z0)
print (2 - 4/3.14159*np.arccos( A ))

0.566412093931


In [5]:
print 3.1415 / 4. * 2.8 / 2

1.099525


In [7]:
print 1.099 * 3.14

3.45086
